In [1]:
import pandas as pd

df_train = pd.read_pickle('../data/train.pkl')
df_test = pd.read_pickle('../data/test.pkl')
df_train.index = range(len(df_train))
doc_info = pd.read_pickle('../data/doc_info.pkl')
user_info = pd.read_pickle('../data/user_info.pkl')

In [2]:
del df_train['dt']
del df_train['date']
del df_train['duration']
del df_train['position']

del df_test['dt']
del df_test['date']

del doc_info['title']

In [3]:
doc_info['docid'] = doc_info['docid'].astype('int')

In [6]:
from tqdm import tqdm
keywords_all=[]
for i in tqdm(doc_info['keyword']):
#     print(i)
    key_one=[]
    splts = i.split(',')
    for sp in splts:
        split_=sp.split(':')
        try:
            if float(split_[1])>10:
                key_one.append(split_[0])
        except:
            continue
#     print(key_one)
    keywords_all.append(key_one)

100%|██████████| 347466/347466 [00:09<00:00, 35338.46it/s]


In [7]:
print(keywords_all[0:5])

[[], [], ['初恋', '妆容', '抹胸', '清纯', '珍珠', '珍珠^^吊坠', '直播', '车展', '连衣裙'], ['主播', '清纯', '直播', '车展'], ['交通^^秩序', '牌照', '眼线']]


In [8]:
import gensim.corpora as corpora
id2word = corpora.Dictionary(keywords_all)     # 创建词典
id2word.filter_extremes(no_below=5, no_above=0.5, keep_n=10000) # 出现次数至少为3，至少在一半文档中出现，词典大小最多3000个词

# id2word.save_as_text("dictionary")                   # 保存词典


In [20]:
keywords_all_id=[id2word.doc2bow(t) for t in keywords_all]

In [24]:
print(keywords_all_id[500])

[(313, 1), (1153, 1), (1154, 1), (1155, 1)]


In [29]:
def get_features(corpus_one):
    new_feature=[0 for k in range(10000)]#3000是词典的维度
    for j in corpus_one:
        new_feature[j[0]]=j[1]#tf-idf的j形如(2, 0.17541578348079775)
    s=''
    for i in new_feature:
        s+=str(i)
    return s

keywords_all_feature=[]
for i in tqdm(keywords_all_id):
    keywords_all_feature.append(get_features(i))

100%|██████████| 347466/347466 [20:24<00:00, 283.75it/s]


In [32]:
doc_info['keyword'] = keywords_all_feature

In [34]:
df_train.to_pickle('../data/train0.pkl')
df_test.to_pickle('../data/test0.pkl')
doc_info.to_pickle('../data/doc_info0.pkl')
user_info.to_pickle('../data/user_info0.pkl')

In [1]:
import pandas as pd
df_train = pd.read_pickle('../data/train0.pkl')
df_test = pd.read_pickle('../data/test0.pkl')
doc_info = pd.read_pickle('../data/doc_info0.pkl')
user_info = pd.read_pickle('../data/user_info0.pkl')

In [2]:
df_train_user=pd.merge(df_train,user_info,on='userid')
df_train_user_doc=pd.merge(df_train_user,doc_info,on='docid')

In [3]:
df_test_user=pd.merge(df_test,user_info,on='userid')
df_test_user_doc=pd.merge(df_test_user,doc_info,on='docid')

In [4]:
df_test_user_doc.sort_values("id",inplace=True)
df_test_user_doc.index = range(len(df_test_user_doc))
del df_test_user_doc['id']

In [14]:
from tqdm import tqdm
import numpy as np
for i in df_train_user_doc.columns:
    print(i)
    for index,j in tqdm(enumerate(df_train_user_doc[i])):
        if np.isnan(j):
            print(df_train_user_doc[i].mean())
            df_train_user_doc[i][index]=df_train_user_doc[i].mean()

    

userid


816711it [00:01, 522647.03it/s]


docid


816711it [00:01, 514479.82it/s]


timestamp


...

category2nd


816711it [00:01, 534177.38it/s]


keyword


0it [00:00, ?it/s]


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
# import numpy as np
# print(df_train_user_doc.describe())
# # 数据集概览
# print(df_train_user_doc.head(5))
# # 前5行


In [ ]:
# from tqdm import tqdm
# for i in tqdm(df_train_user_doc.columns):
#     print(i)
#     df_train_user_doc[i].fillna((df_train_user_doc[i].mean()), inplace=True)

In [ ]:
# for i in tqdm(df_test_user_doc.columns):
#     print(i)
#     df_test_user_doc[i].fillna((df_test_user_doc[i].mean()), inplace=True)

In [43]:
from sklearn.model_selection import train_test_split
x_columns = [x for x in df_train_user_doc.columns if x not in ['click','dt','date','duration','position']]
X = df_train_user_doc[x_columns]      # 样本
y = df_train_user_doc['click']    # 标签

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=9)


In [44]:
from sklearn.ensemble import GradientBoostingClassifier
gbm0 = GradientBoostingClassifier(random_state=9)
gbm0.fit(x_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
from sklearn.metrics import roc_auc_score,accuracy_score
print("tr-accuracy: %.4g" % accuracy_score(y_train.values, gbm0.predict(x_train)))      # Accuracy : 0.9855
print("tr-AUC: %f" % roc_auc_score(y_train, gbm0.predict_proba(x_train)[:, 1]))      # AUC Score (Train): 0.910597

print("val-accuracy: %.4g" % accuracy_score(y_val, gbm0.predict(x_val)))      # Accuracy : 0.9825
print("val-AUC: %f" % roc_auc_score(y_val, gbm0.predict_proba(x_val)[:, 1]))      # AUC Score (Train): 0.827217

In [ ]:
x_columns_0 = [x for x in df_test.columns if x not in ['click','dt','date','duration','position']]
X_0 = df_test[x_columns_0]      # 样本
print(X_0[0:1])

In [ ]:
y_pre_0=gbm0.predict_proba(X_0)[:, 1]

In [18]:
def store_list(lsit,text_path):
    ff = open(text_path, encoding='utf-8', mode='w')
    for line_list in lsit:
        ff.write(str(line_list))  # 写入一个新文件中
        ff.write("\n")
store_list(keyword_map_list,'../data/keyword_map_list.txt')